# Parse SEC filings

In [7]:
from __future__ import division, unicode_literals 

import pandas as pd
import numpy as np
import time, sys, os, re
import datetime, pickle, bz2
import getpass
from tqdm import tqdm
import datetime
from datetime import datetime
import string
# Import BeautifulSoup
from bs4 import BeautifulSoup
import re

In [549]:
def cleanStr(test_df_1):
    test_df_1 = test_df_1.translate(str.maketrans('', '', string.punctuation))
    test_df_1 = re.sub('&#160;',' ', test_df_1) 
    test_df_1 = re.sub('&nbsp;', ' ', test_df_1) 
    test_df_1 = re.sub('\xa0', ' ', test_df_1)
    test_df_1 = re.sub('\n',' ', test_df_1)
    test_df_1 = re.sub(':',' ', test_df_1)
    test_df_1 = re.sub("–", ' ', test_df_1)
    test_df_1 = re.sub('\t', ' ', test_df_1)
    test_df_1 = re.sub('·', ' ', test_df_1)
    test_df_1 = re.sub(' ·', ' ', test_df_1)
    test_df_1 = re.sub('     ', ' ', test_df_1)
    test_df_1 = re.sub('    ', ' ', test_df_1)
    test_df_1 = re.sub('   ', ' ', test_df_1)
    test_df_1 = re.sub('  ', ' ', test_df_1)
        
    return test_df_1

In [82]:
def cleanDf(test_df):
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)
    test_df.replace('   ',' ',regex=True,inplace=True)
    test_df.replace('  ',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    
    return test_df

# unit test

In [396]:
ma_list = sorted(os.listdir("/home/dada/CyberSpace/GQ/AI_prj/10-K/{}".format('Pos')))
ma_list = [i for i in ma_list if ".html" in i]

In [543]:
with open(f"/home/dada/CyberSpace/GQ/AI_prj/10-K/Pos/FTFT.html") as fp:
    document = BeautifulSoup(fp.read()).get_text().lower()

In [552]:
parse_i_j = BeautifulSoup(document, 'lxml')
parse_i_j = parse_i_j.get_text("\n\n")
parse_i_j = cleanStr(parse_i_j)

In [553]:
"item 7" in parse_i_j

True

In [554]:
#regex = re.compile(r'(item\s7\smanagements discussion)|(item\s(7A|7|8))', re.IGNORECASE)

In [555]:
matches = regex.finditer(parse_i_j)
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df

,item,start,end
0,item 7 managements discussion,4140,4169
1,item 8 financial statements,4305,4332
2,item 7 managements discussion,121072,121101
3,item 8 financial statements,171462,171489


In [540]:
# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

# Display the dataframe
pos_dat.reset_index(drop = True, inplace = True)
pos_dat

,item,start,end
0,item 7 managements discussion,37786,37815
1,item 8 financial statements,65821,65848


# parse SEC 10-K in Batch model 

In [556]:
def prepText_K(ma_status = "Pos"):
    
    ma_dict = {} 
    
    ma_list = sorted(os.listdir("/home/dada/CyberSpace/GQ/AI_prj/10-K/{}".format(ma_status)))
    ma_list = [i for i in ma_list if ".html" in i]
    
    for i in tqdm(ma_list):
        
        ticker_i = i.replace(".html", "")    
        with open(f"/home/dada/CyberSpace/GQ/AI_prj/10-K/{ma_status}/{i}") as fp:
            document = BeautifulSoup(fp.read()).get_text().lower()
            
#            if "item 2" in document:
#             regex_i_j = regex.finditer(document)
#             match_df = pd.DataFrame([(x.start(), x.end()) for x in regex_i_j])
#             parse_i_j = BeautifulSoup(document[match_df.iloc[0,1]:match_df.iloc[len(match_df)-1,0]], 'lxml')

            parse_i_j = BeautifulSoup(document, 'lxml')
            parse_i_j = parse_i_j.get_text("\n\n")
            parse_i_j = cleanStr(parse_i_j)

#             matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(parse_i_j)])
#             matches_array.columns = ['SearchTerm', 'Start']
                        
            #regex = re.compile(r'(>item(\s| | )(7A|7|8)\.{0,1})|(item\s(7A|7|8))', re.IGNORECASE)
            regex = re.compile(r'item\s7[ ]{0,1}\smanagement(.|)s\sdiscussion|' 'item\s8[ ]{0,1}(consolidated\sfinancial|financial)\sstatements',re.IGNORECASE)
            matches = regex.finditer(parse_i_j)

            test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
            
            if len(test_df) == 0:
                print(f"{ticker_i} is not processed")
                pass
            
            else:
                test_df.columns = ['item', 'start', 'end']            
                test_df = cleanDf(test_df)
                
                if "item8" in [i[:5] for i in test_df.item]:
                    test_df = test_df[:max(list(test_df[['item8' in i for i in test_df.item]].index+1))]
            
                    pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')
                    pos_dat.reset_index(drop = True, inplace = True)
                    ma_dict[ticker_i] = parse_i_j[int(pos_dat.start[0]):int(pos_dat.start[len(pos_dat)-1])]
                else:
                    print(f"{ticker_i} is not processed")
                    pass
            
    return ma_dict

# extract Pos text for training

In [557]:
%%time 

ma_pos_10K = prepText_K(ma_status = "Pos")

 16%|█▋        | 91/553 [02:51<31:03,  4.03s/it]

C is not processed


 30%|███       | 167/553 [05:10<09:42,  1.51s/it]

ECPG is not processed


 42%|████▏     | 232/553 [07:45<12:29,  2.33s/it]

GTLS is not processed


 43%|████▎     | 240/553 [08:01<15:24,  2.95s/it]

HIG is not processed


100%|██████████| 553/553 [18:01<00:00,  1.96s/it]

CPU times: user 17min 59s, sys: 2.51 s, total: 18min 2s
Wall time: 18min 1s


In [558]:
ma_pos_10K_df = pd.DataFrame.from_dict(ma_pos_10K, orient ='index')

In [559]:
print(ma_pos_10K_df.shape)
ma_pos_10K_df.head()

(549, 1)


,0
AACI,item 7 management’s discussion and analysis of...
AAL,item 7 management’s discussion and analysis of...
AAP,item 7 management’s discussion and analysis of...
ABC,item 7 managements discussion and analysis of ...
ACEV,item 7 management’s discussion and analysis of...


In [560]:
pickle.dump(ma_pos_10K_df.reset_index(), open("ma_pos_10K_new.pkl", "wb"))

# extract Neg text for training

In [ ]:
%%time 

ma_neg_10K = prepText_K(ma_status = "Neg")

  5%|▍         | 27/551 [01:00<14:58,  1.71s/it]

In [ ]:
ma_neg_10K_df = pd.DataFrame.from_dict(ma_neg_10K, orient ='index')

In [ ]:
print(ma_neg_10K_df.shape)
ma_neg_10K_df.head()

In [ ]:
pickle.dump(ma_neg_10K_df.reset_index(), open("ma_neg_10K_new.pkl", "wb"))

# extract Neg text for testing

In [ ]:
%%time 

tst_neg_10K = prepText_K(ma_status = "Tst/Neg")

In [ ]:
tst_neg_10K_df = pd.DataFrame.from_dict(tst_neg_10K, orient ='index')

In [ ]:
print(tst_neg_10K_df.shape)
tst_neg_10K_df.head()

In [ ]:
pickle.dump(tst_neg_10K_df.reset_index(), open("tst_neg_10K_new.pkl", "wb"))

# extract Neg text

In [ ]:
%%time 

tst_pos_10K = prepText_K(ma_status = "Tst/Pos")

In [ ]:
tst_pos_10K_df = pd.DataFrame.from_dict(tst_pos_10K, orient ='index')

In [ ]:
print(tst_pos_10K_df.shape)
tst_pos_10K_df.head()

In [ ]:
pickle.dump(tst_pos_10K_df.reset_index(), open("tst_pos_10K_new.pkl", "wb"))